In [ ]:
!pip install transformers==4.32.0 peft==0.5.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes
#!pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui
!pip install accelerate
!pip install datasets

In [6]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0

In [ ]:
base_model = "NousResearch/Llama-2-13b-hf"
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_8bit = True,)
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()


In [4]:
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]


In [ ]:
#tokens = tokenizer(prompt, return_tensors='pt', padding=True)
#res = model.generate(**tokens, max_length=512)
#res_sentences = [tokenizer.decode(i) for i in res]
#out_text = [o.split("Answer: ")[1] for o in res_sentences]

In [ ]:
for sentiment in out_text:
   print(sentiment)

In [ ]:
from datasets import list_datasets, load_dataset
dataset = load_dataset('zeroshot/twitter-financial-news-sentiment', split='validation[:2%]')

In [8]:
prompt_raw = dataset['text'][0:5]

In [10]:
prompt2 = []
for i in prompt_raw:
  prompt2_i = '''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive} \n Input: ''' + i + '''\n Answer: '''
  prompt2.append(prompt2_i)

In [ ]:
tokens2 = tokenizer(prompt2, return_tensors='pt', padding=True, max_length=512)
res2 = model.generate(**tokens2,  max_length=512)
res_sentences2 = [tokenizer.decode(i) for i in res2]
out_text2 = [o.split("Answer: ")[1] for o in res_sentences2]

In [ ]:
for sentiment in out_text2:
   print(sentiment)

In [ ]:
for p in prompt2:
   print(p)

In [1]:
import requests

def query(payload, model_id, api_token):
	headers = {"Authorization": f"Bearer {api_token}"}
	API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

model_id = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
#model_fin_id = "FinGPT/fingpt-sentiment_llama2-13b_lora"
api_token = "" # get yours at hf.co/settings/tokens
data = query("The goal of life is [MASK].", model_id, api_token)

In [ ]:
for p in prompt_raw:
  q_p = query(p, model_id, api_token)
  print(q_p)